In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import load_csv_pitch
import numpy as np
from tqdm import tqdm
import datetime
import json

In [2]:
device = (
   "cuda" if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(device)
# device = "cpu"

BATCH_SIZE = 3500
train_loader, test_loader, validation_loader = load_csv_pitch.load_MSD(batch_size=BATCH_SIZE, device=device)

print("Data Loaded")

cuda
Data Loaded


In [3]:
import time
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.conv = nn.Conv1d(1, 6, 9, 1, 4)
        self.maxpool = nn.MaxPool1d(3, 3)

        self.fc1 = nn.Linear(13212, 5000)
        self.fc2 = nn.Linear(5000, 1000)
        self.fc3 = nn.Linear(1000, 12)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv(x)
        x = self.maxpool(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        max_values, _ = torch.max(x, dim = 1, keepdim = True)
        x = x / max_values
        return x
        
model = model().to(device)
# saved_model = torch.load('models/timbre_model_epoch_10.pth')
# model.load_state_dict(saved_model)

print(model)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 20
min_valid_loss = 0.1160474456846714

model(
  (conv): Conv1d(1, 6, kernel_size=(9,), stride=(1,), padding=(4,))
  (maxpool): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=13212, out_features=5000, bias=True)
  (fc2): Linear(in_features=5000, out_features=1000, bias=True)
  (fc3): Linear(in_features=1000, out_features=12, bias=True)
)


In [4]:
#start train loop
for e in range(7, epochs):
    start = time.time()
    start_time = datetime.datetime.now()
    model.train()
    train_loss = 0.0
    validation_loss = 0.0
    
    # actual model training
    for i, (data,labels) in tqdm(enumerate(train_loader)):
        data, labels = data.to(device), labels.to(device)
         
        # Clear the gradients
        optimizer.zero_grad()
        # Forward Pass
        target = model(data.float())
        # Find the Loss
        loss = criterion(target,labels.float())
        # Calculate gradients
        loss.backward()
        # Update Weights
        optimizer.step()
        # Calculate Loss
        train_loss += loss.item()

    train_loss = train_loss / (i + 1)

    # testing valiation loss
    for i, (data,labels) in tqdm(enumerate(validation_loader)):
        data, labels = data.to(device), labels.to(device)
         
        # Forward Pass
        target = model(data.float())
        # Find the validation loss
        loss = criterion(target,labels.float())

        # Calculate Loss
        validation_loss += loss.item()

    validation_loss = validation_loss / (i + 1)
    
    if validation_loss < min_valid_loss:
        print(f'validation loss decreased: {min_valid_loss} -> {validation_loss}')
        print(f'saving current model')
        torch.save(model.state_dict(), f'pitch_model_epoch_{e + 1}.pth')
        min_valid_loss = validation_loss


    end = time.time()
    end_time = datetime.datetime.now()
    print(f"Epoch: {e+1}/{epochs}")
    print(f'Start @ {start_time.hour}:{start_time.minute}, End @ {end_time.hour}:{end_time.minute}')
    print(f"Epoch Duration: {end-start:.2f}s")
    print(f"Training Loss: {train_loss:.6f}")
    print(f"Validation Loss: {validation_loss:.6f}\n")

205it [44:04, 12.90s/it]
42it [03:15,  4.65s/it]


Epoch: 8/20
Start @ 14:5, End @ 14:52
Epoch Duration: 2840.78s
Training Loss: 0.125356
Validation Loss: 0.123532



205it [46:39, 13.65s/it]
42it [02:57,  4.24s/it]


validation loss decreased: 0.1160474456846714 -> 0.10756584789071764
saving current model
Epoch: 9/20
Start @ 14:52, End @ 15:42
Epoch Duration: 2977.66s
Training Loss: 0.108540
Validation Loss: 0.107566



205it [42:15, 12.37s/it]
42it [03:01,  4.32s/it]


...

validation loss decreased: 0.08100792898663453 -> 0.06797928416303226
saving current model
Epoch: 13/20
Start @ 18:1, End @ 18:48
Epoch Duration: 2786.87s
Training Loss: 0.054534
Validation Loss: 0.067979



205it [43:41, 12.79s/it]
42it [03:05,  4.42s/it]


validation loss decreased: 0.06797928416303226 -> 0.06760534014375437
saving current model
Epoch: 14/20
Start @ 18:48, End @ 19:35
Epoch Duration: 2807.36s
Training Loss: 0.052985
Validation Loss: 0.067605



205it [45:22, 13.28s/it]
0it [00:28, ?it/s]


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\So\anaconda3\envs\shazam\lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\Users\So\anaconda3\envs\shazam\lib\site-packages\torch\utils\data\_utils\fetch.py", line 53, in fetch
    data = self.dataset[possibly_batched_index]
  File "c:\Users\So\Documents\code\shazam\load_csv_pitch.py", line 50, in __getitem__
    return  torch.tensor(y).to(self.device), torch.tensor(pitch).to(self.device)
RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

